# Phân tích và Dự đoán Sống sót trên tàu Titanic
**Thư viện cần thiết:**

- `pandas`
- `numpy`
- `matplotlib`, `seaborn`
- `sklearn`

In [17]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Xem thử dữ liệu

In [10]:
try:
    train_df = pd.read_csv('../../data/train_imputed_method2.csv')
    test_df = pd.read_csv('../../data/test_imputed_method2.csv')
except FileNotFoundError:
    print("LỖI: Không tìm thấy file dữ liệu. Hãy đảm bảo bạn có cấu trúc thư mục 'data/...' chính xác.")
    raise

print("Dữ liệu train:")
display(train_df.head())

print("\nDữ liệu test:")
display(test_df.head())

Dữ liệu train:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,HasCabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,0



Dữ liệu test:


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,HasCabin
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs,0


# Chuẩn bị dữ liệu cho mô hình

1.  Tách biến mục tiêu `Survived` (cái chúng ta muốn dự đoán) ra khỏi tập huấn luyện.
2.  Xác định các biến đặc trưng (features) sẽ được dùng làm đầu vào cho mô hình.
3.  Lưu lại `PassengerId` của tập test để dùng cho file kết quả cuối cùng.

In [14]:

# 1. Tự động chọn tất cả các cột có kiểu dữ liệu là số
X_train_numeric = train_df.select_dtypes(include=np.number)
X_test_numeric = test_df.select_dtypes(include=np.number)

# 2. Xác định biến mục tiêu (y)
y_train = train_df['Survived']

# 3. Xác định các biến đặc trưng (X)
# Loại bỏ cột 'Survived' và 'PassengerId' vì chúng không phải là đặc trưng dự đoán
features_to_drop = ['Survived', 'PassengerId']
X_train = X_train_numeric.drop(columns=features_to_drop, errors='ignore')

# 4. Chuẩn bị dữ liệu test
# Lưu lại PassengerId để tạo file submission
test_passenger_ids = test_df['PassengerId']
# Dữ liệu test cũng phải bỏ cột PassengerId
X_test = X_test_numeric.drop(columns=['PassengerId'], errors='ignore')

# Đảm bảo các cột của X_train và X_test khớp nhau
common_cols = list(set(X_train.columns) & set(X_test.columns))
X_train = X_train[common_cols]
X_test = X_test[common_cols]


print("Các cột được sử dụng làm đặc trưng (features):")
print(X_train.columns)
print("\nKích thước của X_train:", X_train.shape)
print("Kích thước của y_train:", y_train.shape)
print("Kích thước của X_test:", X_test.shape)

Các cột được sử dụng làm đặc trưng (features):
Index(['Parch', 'Age', 'Pclass', 'Fare', 'HasCabin', 'SibSp'], dtype='object')

Kích thước của X_train: (891, 6)
Kích thước của y_train: (891,)
Kích thước của X_test: (418, 6)


# Tách tập dữ liệu huấn luyện thành tập huấn luyện và tập kiểm định (Validation)

- `X_train_split` (80%): Dùng để huấn luyện mô hình.
- `X_val` (20%): Dùng để kiểm tra hiệu suất của mô hình trên dữ liệu mà nó chưa từng thấy.

In [15]:
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)
# stratify=y_train giúp đảm bảo tỷ lệ Sống sót/Không sống sót trong tập train và val là tương đương nhau.

print("Kích thước tập huấn luyện mới:", X_train_split.shape)
print("Kích thước tập kiểm định (validation):", X_val.shape)

Kích thước tập huấn luyện mới: (712, 6)
Kích thước tập kiểm định (validation): (179, 6)


# Xây dựng và Huấn luyện mô hình Logistic Regression

Bây giờ chúng ta sẽ khởi tạo và huấn luyện mô hình trên tập dữ liệu `X_train_split`.

In [16]:
# Khởi tạo mô hình Logistic Regression
# max_iter=1000 để đảm bảo thuật toán có đủ thời gian để hội tụ
log_model = LogisticRegression(max_iter=1000, random_state=42)

# Huấn luyện mô hình
log_model.fit(X_train_split, y_train_split)

print("Mô hình đã được huấn luyện thành công!")

Mô hình đã được huấn luyện thành công!


# Đánh giá hiệu suất của mô hình

Chúng ta sẽ sử dụng tập kiểm định (`X_val`, `y_val`) để xem mô hình dự đoán chính xác đến mức nào.

In [19]:
# Dự đoán trên tập kiểm định
y_pred_val = log_model.predict(X_val)

# Tính toán và in ra độ chính xác
accuracy = accuracy_score(y_val, y_pred_val)
print(f"Độ chính xác trên tập kiểm định: {accuracy * 100:.2f}%")

# In ra báo cáo chi tiết hơn (precision, recall, f1-score)
print("\nBáo cáo phân loại chi tiết:")
# Dòng này bây giờ sẽ hoạt động
print(classification_report(y_val, y_pred_val))

Độ chính xác trên tập kiểm định: 68.72%

Báo cáo phân loại chi tiết:
              precision    recall  f1-score   support

           0       0.70      0.86      0.77       110
           1       0.65      0.41      0.50        69

    accuracy                           0.69       179
   macro avg       0.67      0.63      0.64       179
weighted avg       0.68      0.69      0.67       179



# Tạo file dự đoán cuối cùng

Mô hình đã được đánh giá. Bây giờ, chúng ta sẽ huấn luyện lại nó trên **toàn bộ tập dữ liệu huấn luyện** để mô hình học được nhiều thông tin nhất có thể, sau đó dùng nó để dự đoán trên tập `X_test`.

In [26]:
# Bước 1: Huấn luyện lại mô hình trên toàn bộ tập dữ liệu huấn luyện
print("Huấn luyện mô hình cuối cùng trên toàn bộ dữ liệu train...")
final_model = LogisticRegression(max_iter=1000, random_state=42)
final_model.fit(X_train, y_train)
print("-> Huấn luyện hoàn tất.")

# Bước 2: Dự đoán kết quả cho tập test
print("Dự đoán trên dữ liệu test...")
test_predictions = final_model.predict(X_test)
print("-> Dự đoán hoàn tất.")

# Bước 3: Tải file submission mẫu để lấy cột 'Survived' gốc (dự đoán cơ sở)
# Đường dẫn này cũng phải đi lên 2 cấp
sample_submission_df = pd.read_csv('../../data/gender_submission.csv')
survived_baseline = sample_submission_df['Survived']

# Bước 4: Tạo DataFrame 
submission_comparison = pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived": survived_baseline,      # Cột 'Survived' từ file gender_submission
    "Survived_pred": test_predictions  # Cột dự đoán từ mô hình regression
})

submission = pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived_pred": test_predictions  # Cột dự đoán từ mô hình regression
})

# Bước 5: Lưu file kết quả
submission_comparison.to_csv('../../models/logistic regression/submission_comparison.csv', index=False)
submission.to_csv('../../models/logistic regression/submission.csv', index=False)

print("File được lưu tại: models/logistic regression")
display(submission_comparison.head())

Huấn luyện mô hình cuối cùng trên toàn bộ dữ liệu train...
-> Huấn luyện hoàn tất.
Dự đoán trên dữ liệu test...
-> Dự đoán hoàn tất.
File được lưu tại: models/logistic regression


,PassengerId,Survived,Survived_pred
0,892,0,0
1,893,1,0
2,894,0,0
3,895,0,0
4,896,1,0
